#### Stock Market API Project

In [1]:
import streamlit as st

In [2]:
api_key = st.secrets["API_KEY"]

Symbol Search

In [3]:
import requests
import pandas as pd

url = "https://alpha-vantage.p.rapidapi.com/query"

querystring = {"datatype":"json","keywords":"microsoft","function":"SYMBOL_SEARCH"}

headers = {
	"x-rapidapi-key": "1954377a57msh873ddf8212bbdbcp1ff9c0jsn6df9baa4ee5b",
	"x-rapidapi-host": "alpha-vantage.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

data = response.json()["bestMatches"]

res = pd.DataFrame(data)

res

,1. symbol,2. name,3. type,4. region,5. marketOpen,6. marketClose,7. timezone,8. currency,9. matchScore
0,MSF0.FRK,MICROSOFT CORP. CDR,Equity,Frankfurt,08:00,20:00,UTC+02,EUR,0.6429
1,MSFT,Microsoft Corporation,Equity,United States,09:30,16:00,UTC-04,USD,0.6154
2,0QYP.LON,Microsoft Corporation,Equity,United Kingdom,08:00,16:30,UTC+01,USD,0.6000
3,MSF.DEX,Microsoft Corporation,Equity,XETRA,08:00,20:00,UTC+02,EUR,0.6000
4,MSF.FRK,Microsoft Corporation,Equity,Frankfurt,08:00,20:00,UTC+02,EUR,0.6000
5,MSFT34.SAO,Microsoft Corporation,Equity,Brazil/Sao Paolo,10:00,17:30,UTC-03,BRL,0.6000
6,MSFT.TRT,Microsoft CDR (CAD Hedged),Equity,Toronto,09:30,16:00,UTC-05,CAD,0.5143


### Getting acutal data for SBIN.BSE

In [4]:
url = "https://alpha-vantage.p.rapidapi.com/query"

querystring = {
    "function": "TIME_SERIES_DAILY",
    "symbol": "SBIN.BSE",
    "outputsize": "compact",
    "datatype": "json",
}

headers = {
    "x-rapidapi-key": api_key,
    "x-rapidapi-host": "alpha-vantage.p.rapidapi.com",
}

response = requests.get(url, headers=headers, params=querystring)

data2 = response.json()["Time Series (Daily)"]
res2 = pd.DataFrame(data2).T

# Convert output to float
res2 = res2.astype(float)

# Conert to index to datetime
res2.index = pd.to_datetime(res2.index)

res2

,1. open,2. high,3. low,4. close,5. volume
2025-05-09,755.25,781.40,755.25,779.40,375997.0
2025-05-08,778.00,786.50,765.00,768.75,463248.0
2025-05-07,762.25,782.55,762.25,776.15,581175.0
2025-05-06,792.20,799.00,771.15,774.15,761282.0
2025-05-05,784.95,799.10,783.65,790.00,771587.0
...,...,...,...,...,...
2024-12-17,859.75,862.95,847.15,850.20,143409.0
2024-12-16,859.00,866.20,857.80,860.70,252691.0
2024-12-13,853.70,863.00,835.00,861.70,455324.0
2024-12-12,862.50,864.20,851.25,853.75,179011.0


In [5]:
res2.dtypes

1. open      float64
2. high      float64
3. low       float64
4. close     float64
5. volume    float64
dtype: object

In [6]:
res2.index


DatetimeIndex(['2025-05-09', '2025-05-08', '2025-05-07', '2025-05-06',
               '2025-05-05', '2025-05-02', '2025-04-30', '2025-04-29',
               '2025-04-28', '2025-04-25', '2025-04-24', '2025-04-23',
               '2025-04-22', '2025-04-21', '2025-04-17', '2025-04-16',
               '2025-04-15', '2025-04-11', '2025-04-09', '2025-04-08',
               '2025-04-07', '2025-04-04', '2025-04-03', '2025-04-02',
               '2025-04-01', '2025-03-28', '2025-03-27', '2025-03-26',
               '2025-03-25', '2025-03-24', '2025-03-21', '2025-03-20',
               '2025-03-19', '2025-03-18', '2025-03-17', '2025-03-13',
               '2025-03-12', '2025-03-11', '2025-03-10', '2025-03-07',
               '2025-03-06', '2025-03-05', '2025-03-04', '2025-03-03',
               '2025-02-28', '2025-02-27', '2025-02-25', '2025-02-24',
               '2025-02-21', '2025-02-20', '2025-02-19', '2025-02-18',
               '2025-02-17', '2025-02-14', '2025-02-13', '2025-02-12',
      

In [7]:
res2.to_csv("SBIN.csv")


In [8]:
import plotly.graph_objects as go


In [9]:
fig = go.Figure(
    data=[
        go.Candlestick(
            x=res2.index,
            open=res2["1. open"],
            high=res2["2. high"],
            low=res2["3. low"],
            close=res2["4. close"],
        )
    ]
)

fig.update_layout({"width": 1000, "height": 800})

fig.show()

## Create a class to show data / plot for any stocks

In [10]:
class StockAPI:

    def __init__(self):
        self.api_key = st.secrets["API_KEY"]
        self.url = "https://alpha-vantage.p.rapidapi.com/query"
        self.headers = {
            "x-rapidapi-key": self.api_key,
            "x-rapidapi-host": "alpha-vantage.p.rapidapi.com",
        }

    def symbol_search(self, company: str) -> pd.DataFrame:
        querystring = {
            "datatype": "json",
            "keywords": company,
            "function": "SYMBOL_SEARCH",
        }
        response = requests.get(url=self.url, headers=self.headers, params=querystring)
        data = response.json()["bestMatches"]
        return pd.DataFrame(data)

    def stock_data(self, symbol: str) -> pd.DataFrame:
        querystring = {
            "function": "TIME_SERIES_DAILY",
            "symbol": symbol,
            "outputsize": "compact",
            "datatype": "json",
        }
        response = requests.get(url=self.url, headers=self.headers, params=querystring)
        data2 = response.json()["Time Series (Daily)"]
        res2 = pd.DataFrame(data2).T

        # Convert output to float
        res2 = res2.astype(float)

        # Conert to index to datetime
        res2.index = pd.to_datetime(res2.index)

        return res2
    
    def plot_chart(self, df: pd.DataFrame) -> go.Figure:
        fig = go.Figure(
            data=[
                go.Candlestick(
                    x=df.index,
                    open=df["1. open"],
                    high=df["2. high"],
                    low=df["3. low"],
                    close=df["4. close"],
                )
            ]
        )

        fig.update_layout({"width": 1000, "height": 800})
        return fig

In [11]:
client = StockAPI()

In [12]:
client.symbol_search("HDFC Bank")


,1. symbol,2. name,3. type,4. region,5. marketOpen,6. marketClose,7. timezone,8. currency,9. matchScore
0,HDB,HDFC Bank Ltd,Equity,United States,09:30,16:00,UTC-04,USD,0.8182
1,HDFCBANK.BSE,HDFC Bank Limited,Equity,India/Bombay,09:15,15:30,UTC+5.5,INR,0.8000
2,H1DB34.SAO,HDFC Bank Limited,Equity,Brazil/Sao Paolo,10:00,17:30,UTC-03,BRL,0.6923
3,HDFA.FRK,HDFC Bank Limited,Equity,Frankfurt,08:00,20:00,UTC+02,EUR,0.6923


In [13]:
client.symbol_search("Tesla")


,1. symbol,2. name,3. type,4. region,5. marketOpen,6. marketClose,7. timezone,8. currency,9. matchScore
0,TSLA,Tesla Inc,Equity,United States,09:30,16:00,UTC-04,USD,0.8889
1,TL0.DEX,Tesla Inc,Equity,XETRA,08:00,20:00,UTC+02,EUR,0.7143
2,TL0.FRK,Tesla Inc,Equity,Frankfurt,08:00,20:00,UTC+02,EUR,0.7143
3,TSLA34.SAO,Tesla Inc,Equity,Brazil/Sao Paolo,10:00,17:30,UTC-03,BRL,0.7143
4,TSLA.TRT,Tesla CDR (CAD Hedged),Equity,Toronto,09:30,16:00,UTC-05,CAD,0.6667
5,TL01.FRK,TESLA INC. CDR DL-001,Equity,Frankfurt,08:00,20:00,UTC+02,EUR,0.3846


In [14]:
df2 = client.stock_data("HDFCBANK.BSE")
df2

,1. open,2. high,3. low,4. close,5. volume
2025-05-09,1905.15,1919.00,1886.45,1889.20,107594.0
2025-05-08,1955.00,1955.00,1918.10,1926.30,331040.0
2025-05-07,1930.60,1948.65,1922.30,1944.65,123501.0
2025-05-06,1935.95,1939.45,1924.00,1930.55,172594.0
2025-05-05,1938.80,1952.00,1933.00,1936.15,268311.0
...,...,...,...,...,...
2024-12-18,1832.50,1835.65,1807.45,1811.20,102823.0
2024-12-17,1856.05,1862.25,1826.35,1832.85,104055.0
2024-12-16,1863.00,1870.00,1855.10,1864.80,133759.0
2024-12-13,1855.20,1874.55,1829.00,1872.05,228126.0


In [15]:
client.plot_chart(df2)

In [17]:
df3 = client.stock_data("TSLA")
df3

,1. open,2. high,3. low,4. close,5. volume
2025-05-09,290.210,307.0400,290.0000,298.26,132387835.0
2025-05-08,279.630,289.8000,279.4100,284.82,97539448.0
2025-05-07,276.880,277.9200,271.0000,276.22,71882408.0
2025-05-06,273.105,277.7300,271.3500,275.35,76715792.0
2025-05-05,284.570,284.8490,274.4000,280.26,94618882.0
...,...,...,...,...,...
2024-12-19,451.880,456.3600,420.0200,436.17,118566146.0
2024-12-18,466.495,488.5399,427.0100,440.13,149340788.0
2024-12-17,475.900,483.9900,457.5101,479.86,131222978.0
2024-12-16,441.090,463.1900,436.1500,463.02,114083811.0


In [ ]:
client.plot_chart(df3)